In [1]:
! pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/1.3 MB ? eta -:--:--


     ━━━━━━━━━━━━━━━━━━━━━━╸━━━━━━━━━━━━━━━━━ 0.7/1.3 MB 22.7 MB/s eta 0:00:01


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.2 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [2]:
MAX_LEN = 128
BATCH_SIZE = 8 # per TPU core
TOTAL_STEPS = 2000  # thats approx 4 epochs
EVALUATE_EVERY = 200
LR =  1e-5

PRETRAINED_MODEL = 'jplu/tf-xlm-roberta-large'
D = '/kaggle/input/jigsaw-multilingual-toxic-comment-classification/'


import os
import numpy as np
import pandas as pd
import tensorflow as tf
print(tf.__version__)
from tensorflow.keras.optimizers import Adam
import transformers
from transformers import TFAutoModelWithLMHead, AutoTokenizer
import logging
# no extensive logging 
logging.getLogger().setLevel(logging.NOTSET)

AUTO = tf.data.experimental.AUTOTUNE

E0000 00:00:1725993393.264808      77 common_lib.cc:798] Could not set metric server port: INVALID_ARGUMENT: Could not find SliceBuilder port 8471 in any of the 0 ports provided in `tpu_process_addresses`="local"
=== Source Location Trace: ===
learning/45eac/tfrc/runtime/common_lib.cc:479
D0910 18:36:33.272946284      77 config.cc:196]                        gRPC EXPERIMENT call_status_override_on_cancellation   OFF (default:OFF)
D0910 18:36:33.272961149      77 config.cc:196]                        gRPC EXPERIMENT call_v3                                OFF (default:OFF)
D0910 18:36:33.272964286      77 config.cc:196]                        gRPC EXPERIMENT canary_client_privacy                  ON  (default:ON)
D0910 18:36:33.272966677      77 config.cc:196]                        gRPC EXPERIMENT capture_base_context                   ON  (default:ON)
D0910 18:36:33.272969145      77 config.cc:196]                        gRPC EXPERIMENT client_idleness                        ON  (defau

2.16.1


/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def connect_to_TPU():
    """Detect hardware, return appropriate distribution strategy"""
    try:
        # TPU detection. No parameters necessary if TPU_NAME environment variable is
        # set: this is always the case on Kaggle.
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        print('Running on TPU ', tpu.master())
    except ValueError:
        tpu = None

    if tpu:
        tf.config.experimental_connect_to_cluster(tpu)
        tf.tpu.experimental.initialize_tpu_system(tpu)
        strategy = tf.distribute.experimental.TPUStrategy(tpu)
    else:
        # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
        strategy = tf.distribute.get_strategy()

    global_batch_size = BATCH_SIZE * strategy.num_replicas_in_sync

    return tpu, strategy, global_batch_size


tpu, strategy, global_batch_size = connect_to_TPU()
print("REPLICAS: ", strategy.num_replicas_in_sync)

Running on TPU  
INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: local


I0000 00:00:1725993424.212617      77 service.cc:145] XLA service 0x563c462aa130 initialized for platform TPU (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1725993424.212667      77 service.cc:153]   StreamExecutor device (0): TPU, 2a886c8
I0000 00:00:1725993424.212671      77 service.cc:153]   StreamExecutor device (1): TPU, 2a886c8
I0000 00:00:1725993424.212674      77 service.cc:153]   StreamExecutor device (2): TPU, 2a886c8
I0000 00:00:1725993424.212677      77 service.cc:153]   StreamExecutor device (3): TPU, 2a886c8
I0000 00:00:1725993424.212679      77 service.cc:153]   StreamExecutor device (4): TPU, 2a886c8
I0000 00:00:1725993424.212682      77 service.cc:153]   StreamExecutor device (5): TPU, 2a886c8
I0000 00:00:1725993424.212684      77 service.cc:153]   StreamExecutor device (6): TPU, 2a886c8
I0000 00:00:1725993424.212688      77 service.cc:153]   StreamExecutor device (7): TPU, 2a886c8


INFO:tensorflow:Finished initializing TPU system.


INFO:tensorflow:Found TPU system:


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


REPLICAS:  8


In [4]:

D_TRANS = '/kaggle/input/jigsaw-train-multilingual-coments-google-api/'

def load_jigsaw_trans(langs=['tr','it','es','ru','fr','pt'], 
                      columns=['comment_text', 'toxic']):
    train_6langs=[]
    for i in range(len(langs)):
        fn = D_TRANS+'jigsaw-toxic-comment-train-google-%s-cleaned.csv'%langs[i]
        train_6langs.append(downsample(pd.read_csv(fn)[columns]))

    return train_6langs

def downsample(df):
    """Subsample the train dataframe to 50%-50%"""
    ds_df= pd.concat([
        df.query('toxic==1'),
        df.query('toxic==0').sample(sum(df.toxic))
    ])
    
    return ds_df
    

train_df = pd.concat(load_jigsaw_trans()) 


In [5]:
%%time

def regular_encode(texts, tokenizer, maxlen=512):
    enc_di = tokenizer.batch_encode_plus(
        texts.tolist(),  # Convert NumPy array to list
        return_attention_mask=False,  
        return_token_type_ids=False,
        pad_to_max_length=True,
        max_length=maxlen
    )
    
    return np.array(enc_di['input_ids'])

tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_MODEL)
X_train = regular_encode(train_df.comment_text.values, tokenizer, maxlen=MAX_LEN)
# X_val = regular_encode(val_df.comment_text.values, tokenizer, maxlen=MAX_LEN)
# X_test = regular_encode(test_df.content.values, tokenizer, maxlen=MAX_LEN)


DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443


DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /jplu/tf-xlm-roberta-large/resolve/main/tokenizer_config.json HTTP/11" 404 0


DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /jplu/tf-xlm-roberta-large/resolve/main/config.json HTTP/11" 200 0


DEBUG:filelock:Attempting to acquire lock 134132173918640 on /root/.cache/huggingface/hub/.locks/models--jplu--tf-xlm-roberta-large/3acc6bc7228b5c03c361e36291bc513a51090e40.lock


DEBUG:filelock:Lock 134132173918640 acquired on /root/.cache/huggingface/hub/.locks/models--jplu--tf-xlm-roberta-large/3acc6bc7228b5c03c361e36291bc513a51090e40.lock


...

DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /jplu/tf-xlm-roberta-large/resolve/main/special_tokens_map.json HTTP/11" 404 0


/usr/local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


/usr/local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2870: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


CPU times: user 12min 50s, sys: 37 s, total: 13min 27s
Wall time: 31.8 s


In [6]:
def prepare_mlm_input_and_labels(X):
    # 15% BERT masking
    inp_mask = np.random.rand(*X.shape)<0.15 
    # do not mask special tokens
    inp_mask[X<=2] = False
    # set targets to -1 by default, it means ignore
    labels =  -1 * np.ones(X.shape, dtype=int)
    # set labels for masked tokens
    labels[inp_mask] = X[inp_mask]
    
    # prepare input
    X_mlm = np.copy(X)
    # set input to [MASK] which is the last token for the 90% of tokens
    # this means leaving 10% unchanged
    inp_mask_2mask = inp_mask  & (np.random.rand(*X.shape)<0.90)
    X_mlm[inp_mask_2mask] = 250001  # mask token is the last in the dict

    # set 10% to a random token
    inp_mask_2random = inp_mask_2mask  & (np.random.rand(*X.shape) < 1/9)
    X_mlm[inp_mask_2random] = np.random.randint(3, 250001, inp_mask_2random.sum())
    
    return X_mlm, labels



X_train_mlm, y_train_mlm = prepare_mlm_input_and_labels(X_train)

In [7]:
def create_dist_dataset(X, y=None, training=False):
    dataset = tf.data.Dataset.from_tensor_slices(X)

    ### Add y if present ###
    if y is not None:
        dataset_y = tf.data.Dataset.from_tensor_slices(y)
        dataset = tf.data.Dataset.zip((dataset, dataset_y))
        
    ### Repeat if training ###
    if training:
        dataset = dataset.shuffle(len(X)).repeat()

    dataset = dataset.batch(global_batch_size).prefetch(AUTO)

    ### make it distributed  ###
    dist_dataset = strategy.experimental_distribute_dataset(dataset)

    return dist_dataset
    
    
train_dist_dataset = create_dist_dataset(X_train_mlm, y_train_mlm, True)

In [8]:
%%time

def create_mlm_model_and_optimizer():
    with strategy.scope():
        model = TFAutoModelWithLMHead.from_pretrained(PRETRAINED_MODEL)
        optimizer = tf.keras.optimizers.Adam(learning_rate=LR)
    return model, optimizer


mlm_model, optimizer = create_mlm_model_and_optimizer()
mlm_model.summary()

/usr/local/lib/python3.10/site-packages/transformers/models/auto/modeling_tf_auto.py:721: FutureWarning: The class `TFAutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `TFAutoModelForCausalLM` for causal language models, `TFAutoModelForMaskedLM` for masked language models and `TFAutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /jplu/tf-xlm-roberta-large/resolve/main/config.json HTTP/11" 200 0


DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /jplu/tf-xlm-roberta-large/resolve/main/model.safetensors HTTP/11" 404 0


DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /jplu/tf-xlm-roberta-large/resolve/main/tf_model.h5 HTTP/11" 302 0


DEBUG:filelock:Attempting to acquire lock 134144582578336 on /root/.cache/huggingface/hub/.locks/models--jplu--tf-xlm-roberta-large/bd502c50deb6efcb00fd91768ac2a933716d50b8d85b1fcad21b7d3116ed25c7.lock


DEBUG:filelock:Lock 134144582578336 acquired on /root/.cache/huggingface/hub/.locks/models--jplu--tf-xlm-roberta-large/bd502c50deb6efcb00fd91768ac2a933716d50b8d85b1fcad21b7d3116ed25c7.lock


...

Total params: 560142482 (2.09 GB)


Trainable params: 560142482 (2.09 GB)


Non-trainable params: 0 (0.00 Byte)


_________________________________________________________________


CPU times: user 51.6 s, sys: 1min 18s, total: 2min 9s
Wall time: 1min 17s


In [9]:
def define_mlm_loss_and_metrics():
    with strategy.scope():
        mlm_loss_object = masked_sparse_categorical_crossentropy

        def compute_mlm_loss(labels, predictions):
            per_example_loss = mlm_loss_object(labels, predictions)
            loss = tf.nn.compute_average_loss(
                per_example_loss, global_batch_size = global_batch_size)
            return loss

        train_mlm_loss_metric = tf.keras.metrics.Mean()
        
    return compute_mlm_loss, train_mlm_loss_metric


def masked_sparse_categorical_crossentropy(y_true, y_pred):
    y_true_masked = tf.boolean_mask(y_true, tf.not_equal(y_true, -1))
    y_pred_masked = tf.boolean_mask(y_pred, tf.not_equal(y_true, -1))
    loss = tf.keras.losses.sparse_categorical_crossentropy(y_true_masked,
                                                          y_pred_masked,
                                                          from_logits=True)
    return loss

            
            
def train_mlm(train_dist_dataset, total_steps=2000, evaluate_every=200):
    step = 0
    ### Training lopp ###
    for tensor in train_dist_dataset:
        distributed_mlm_train_step(tensor) 
        step+=1

        if (step % evaluate_every == 0):   
            ### Print train metrics ###  
            train_metric = train_mlm_loss_metric.result().numpy()
            print("Step %d, train loss: %.2f" % (step, train_metric))     

            ### Reset  metrics ###
            train_mlm_loss_metric.reset_state()
            
        if step  == total_steps:
            break


@tf.function
def distributed_mlm_train_step(data):
    strategy.run(mlm_train_step, args=(data,))


@tf.function
def mlm_train_step(inputs):
    features, labels = inputs

    with tf.GradientTape() as tape:
        predictions = mlm_model(features, training=True)[0]
        loss = compute_mlm_loss(labels, predictions)

    gradients = tape.gradient(loss, mlm_model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, mlm_model.trainable_variables))

    train_mlm_loss_metric.update_state(loss)
    

compute_mlm_loss, train_mlm_loss_metric = define_mlm_loss_and_metrics()

In [10]:
%%time
train_mlm(train_dist_dataset, TOTAL_STEPS, EVALUATE_EVERY)

DEBUG:absl:`TPUStrategy.run` is called with [args: ((PerReplica:{
  0: <tf.Tensor 'data:0' shape=(8, 128) dtype=int64>,
  1: <tf.Tensor 'data_1:0' shape=(8, 128) dtype=int64>,
  2: <tf.Tensor 'data_2:0' shape=(8, 128) dtype=int64>,
  3: <tf.Tensor 'data_3:0' shape=(8, 128) dtype=int64>,
  4: <tf.Tensor 'data_4:0' shape=(8, 128) dtype=int64>,
  5: <tf.Tensor 'data_5:0' shape=(8, 128) dtype=int64>,
  6: <tf.Tensor 'data_6:0' shape=(8, 128) dtype=int64>,
  7: <tf.Tensor 'data_7:0' shape=(8, 128) dtype=int64>
}, PerReplica:{
  0: <tf.Tensor 'data_8:0' shape=(8, 128) dtype=int64>,
  1: <tf.Tensor 'data_9:0' shape=(8, 128) dtype=int64>,
  2: <tf.Tensor 'data_10:0' shape=(8, 128) dtype=int64>,
  3: <tf.Tensor 'data_11:0' shape=(8, 128) dtype=int64>,
  4: <tf.Tensor 'data_12:0' shape=(8, 128) dtype=int64>,
  5: <tf.Tensor 'data_13:0' shape=(8, 128) dtype=int64>,
  6: <tf.Tensor 'data_14:0' shape=(8, 128) dtype=int64>,
  7: <tf.Tensor 'data_15:0' shape=(8, 128) dtype=int64>
}),)] [kwargs: None]

/usr/local/lib/python3.10/site-packages/tensorflow/python/framework/indexed_slices.py:446: UserWarning: Converting sparse IndexedSlices to a dense Tensor with 256002048 elements. This may consume a large amount of memory.
  warnings.warn(


2024-09-10 18:42:15.495483: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node StatefulPartitionedCall.


I0000 00:00:1725993738.746941     871 tpu_compilation_cache_interface.cc:441] TPU host compilation cache miss: cache_key(31ce53bfedaec854:0:0), session_name()


I0000 00:00:1725993787.191303     871 tpu_compile_op_common.cc:245] Compilation of 31ce53bfedaec854:0:0 with session name  took 48.444303416s and succeeded
I0000 00:00:1725993787.328330     871 tpu_compilation_cache_interface.cc:475] TPU host compilation cache: compilation complete for cache_key(31ce53bfedaec854:0:0), session_name(), subgraph_key(std::string(property.function_name) = "cluster_distributed_mlm_train_step_14513142017370184479", property.function_library_fingerprint = 10213811425343110180, property.mlir_module_fingerprint = 0, property.num_replicas = 8, topology.chip_bounds().x = 2, topology.chip_bounds().y = 2, topology.chip_bounds().z = 1, topology.wrap().x = false, topology.wrap().y = false, topology.wrap().z = false, std::string(property.shapes_prefix) = "", property.guaranteed_constants_size = 0, embedding_partitions_fingerprint = "1688352644216761960")
I0000 00:00:1725993787.328413     871 tpu_compilation_cache_interface.cc:541] After adding entry for key 31ce53bfeda

...

Step 1400, train loss: 1.88


Step 1600, train loss: 1.84


Step 1800, train loss: 1.79


Step 2000, train loss: 1.76


CPU times: user 19min 48s, sys: 2min 16s, total: 22min 5s
Wall time: 18min


In [11]:
mlm_model.save_pretrained('./')

DEBUG:h5py._conv:Creating converter from 5 to 3
